In [ ]:
import os
import sys
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, trange
import scipy.sparse as sp

# Negative sample distribution

In [ ]:
meta_table = pd.read_csv('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/meta_table_9845.csv')
meta_table

In [ ]:
gene_id_dict = dict(zip(meta_table['symbol'],meta_table['unified_id']))
entrez_id_dict = dict(zip(meta_table['entrez_id'],meta_table['unified_id']))
gene_set = set(meta_table['symbol'])

In [ ]:
sl_label = pd.read_csv('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/human_sl_6460.csv')[['unified_id_A','unified_id_B']]
sl_label_all = pd.read_csv('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/human_sl_9845.csv')[['unified_id_A','unified_id_B']]
cut_uni_id = np.load('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/wo_compt/cut_all_uni_id.npy',allow_pickle=True).item()
# sl_label = pd.read_csv('../data/preprocessed_data/human_sl_9845.csv')[['unified_id_A','unified_id_B']]
sl_label['o_A'] = sl_label['unified_id_A'].map(cut_uni_id)
sl_label['o_B'] = sl_label['unified_id_B'].map(cut_uni_id)
sl_label = sl_label[['o_A','o_B']]
sl_label = sl_label.dropna()
sl_label.columns = ['unified_id_A','unified_id_B']
sl_label

In [ ]:
35913-26591

In [ ]:
ssss = set(sl_label['unified_id_A']) | set(sl_label['unified_id_B'])

In [ ]:
sl_label_all['wo_comp'] = [1 if all([sl_label_all.iloc[i,0] in ssss, sl_label_all.iloc[i,1] in ssss]) else 0 for i in range(len(sl_label_all))]
sl_label_all

In [ ]:
sl_label_all[sl_label_all['wo_comp']==0]

In [ ]:
gene_id_set = list(set(sl_label['unified_id_A'])|set(sl_label['unified_id_B']))
len(gene_id_set)

In [ ]:
one_time_neg_index_exp = np.load('../data/preprocessed_data/one_time_neg_index_exp.npy',allow_pickle=True)
five_time_neg_index_exp = np.load('../data/preprocessed_data/five_time_neg_index_exp.npy',allow_pickle=True)
twenty_time_neg_index_exp = np.load('../data/preprocessed_data/twenty_time_neg_index_exp.npy',allow_pickle=True)
fifty_time_neg_index_exp = np.load('../data/preprocessed_data/fifty_time_neg_index_exp.npy',allow_pickle=True)

one_time_neg_index_dep = np.load('../data/preprocessed_data/one_time_neg_index_dep.npy',allow_pickle=True)
five_time_neg_index_dep = np.load('../data/preprocessed_data/five_time_neg_index_dep.npy',allow_pickle=True)
twenty_time_neg_index_dep = np.load('../data/preprocessed_data/twenty_time_neg_index_dep.npy',allow_pickle=True)
fifty_time_neg_index_dep = np.load('../data/preprocessed_data/fifty_time_neg_index_dep.npy',allow_pickle=True)

In [ ]:
DepMap_Effect_corr = pd.read_csv('../data/preprocessed_data/DepMap_Effect_corr.csv',index_col=0)
DepMap_Expression_corr = pd.read_csv('../data/preprocessed_data/DepMap_Expression_corr.csv', index_col=0)

In [ ]:
DepMap_Expression_corr.columns = DepMap_Expression_corr.columns.astype(int)
DepMap_Expression_corr.index = DepMap_Expression_corr.index.astype(int)
DepMap_Expression_corr = DepMap_Expression_corr[range(9845)]
DepMap_Expression_corr = DepMap_Expression_corr.sort_index()
DepMap_Expression_corr

In [ ]:
all_one = []
for i in range(9845):
    if (DepMap_Expression_corr.loc[i,:]==1).all():
        all_one.append(i)
for i in all_one:
    DepMap_Expression_corr.loc[i,:]=np.nan
    DepMap_Expression_corr.loc[:,i]=np.nan

In [ ]:
DepMap_Effect_corr.columns = DepMap_Effect_corr.columns.astype(int)
DepMap_Effect_corr.index = DepMap_Effect_corr.index.astype(int)
DepMap_Effect_corr

In [ ]:
# print(gene_id_set)
gene_id_set = np.asarray(gene_id_set,dtype=int)
sl_set = set(zip(list(sl_label.iloc[:,0].values),list(sl_label.iloc[:,1].values)))
gene_id_set

In [ ]:
1517 in gene_id_set

In [ ]:
from tqdm import tqdm
neg_ids_scores_exp_wo_compt = []
for ind_a in tqdm(gene_id_set):
    for ind_b in gene_id_set:
        if (ind_a,ind_b) in sl_set or (ind_b,ind_a) in sl_set:
            continue
        if ind_a < ind_b:
            pair = (ind_a,ind_b,DepMap_Expression_corr.loc[ind_a,ind_b])
            neg_ids_scores_exp_wo_compt.append(pair)
neg_ids_scores_exp_wo_compt = np.asarray(neg_ids_scores_exp_wo_compt)
sorted_neg_ids_scores_exp_wo_compt = neg_ids_scores_exp_wo_compt[neg_ids_scores_exp_wo_compt[:,2].argsort()]
sorted_neg_ids_scores_exp_wo_compt
print(sorted_neg_ids_scores_exp_wo_compt.shape)

In [ ]:
sorted_neg_ids_scores_exp_wo_compt

In [ ]:
from tqdm import tqdm
neg_ids_scores_dep_wo_compt = []
for ind_a in tqdm(gene_id_set):
    for ind_b in gene_id_set:
        if (ind_a,ind_b) in sl_set or (ind_b,ind_a) in sl_set:
            continue
        if ind_a < ind_b:
            pair = (ind_a,ind_b,DepMap_Effect_corr.loc[int(ind_a),int(ind_b)])
            neg_ids_scores_dep_wo_compt.append(pair)
neg_ids_scores_dep_wo_compt = np.asarray(neg_ids_scores_dep_wo_compt)
sorted_neg_ids_scores_dep_wo_compt = neg_ids_scores_dep_wo_compt[neg_ids_scores_dep_wo_compt[:,2].argsort()]
sorted_neg_ids_scores_dep_wo_compt
print(sorted_neg_ids_scores_dep_wo_compt.shape)

In [ ]:
sorted_neg_ids_scores_dep_wo_compt.shape

In [ ]:
np.save('../data/preprocessed_data/wo_compt/sorted_neg_ids_scores_exp_wo_compt.npy',sorted_neg_ids_scores_exp_wo_compt)
np.save('../data/preprocessed_data/wo_compt/sorted_neg_ids_scores_dep_wo_compt.npy',sorted_neg_ids_scores_dep_wo_compt)

In [ ]:
sorted_neg_ids_scores_exp = np.load('../data/preprocessed_data/sorted_neg_ids_scores_exp.npy')
sorted_neg_ids_scores_dep = np.load('../data/preprocessed_data/sorted_neg_ids_scores_dep.npy')

In [ ]:
sorted_neg_ids_scores_exp

In [ ]:
sorted_neg_ids_scores_dep

In [ ]:
sl_label['exp_score'] = [DepMap_Expression_corr.loc[sl_label.iloc[i,0],sl_label.iloc[i,1]] for i in range(len(sl_label))]
sl_label['dep_score'] = [DepMap_Effect_corr.loc[sl_label.iloc[i,0],sl_label.iloc[i,1]] for i in range(len(sl_label))]
sl_label

In [ ]:
exp_df = pd.DataFrame(columns=['pnr','score'])
times = ['one','five','twenty','fifty','pos']
scores = [one_time_neg_index_exp,five_time_neg_index_exp,twenty_time_neg_index_exp,fifty_time_neg_index_exp,sl_label['exp_score'].values]
col1 = []
col2 = []
for i in range(5):
    tmpcol1 = [times[i]]*len(scores[i])
    if i==4:
        tmpcol2 = list(scores[i])
    else:
        tmpcol2 = list(sorted_neg_ids_scores_exp[scores[i],2])
    col1+=tmpcol1
    col2+=tmpcol2
exp_df['pnr'] = col1
exp_df['score'] = col2
exp_df['score'] = exp_df['score'].astype('float')

In [ ]:


f,ax = plt.subplots(figsize=(7.5, 7.5))

sns.set_theme(style="white")
# Plot a histogram of the correlation
# sns.displot(data=exp_df,x='score', kind = 'kde',multiple='layer',hue = 'pnr', linewidth=0.5,alpha=.8)
# sns.kdeplot(data=np.asarray(sorted_neg_ids_scores_exp[one_time_neg_index_exp,2]), fill=True, legend=False, linewidth=0.5,alpha=.8,color=sns.color_palette("pastel")[0])
sns.kdeplot(data=exp_df,x='score',hue = 'pnr', fill=True,common_norm = False, linewidth=0.5,alpha=.6)
# sns.histplot(data=sl_corr, kde=True)

# plt.hist(sl_corr, bins=100)
plt.xlabel('Gene expression correlation coefficient')
# plt.xlabel('Gene dependency score correlation coefficient')
plt.ylabel('Frequency')
plt.title('Distribution of known SL gene pairs')
plt.xlim(-1,1)
# plt.show()

plt.savefig('sl_corr_dep_03.svg')

In [ ]:
dep_df = pd.DataFrame(columns=['pnr','score'])
times = ['one','five','twenty','fifty','pos']
scores = [one_time_neg_index_dep,five_time_neg_index_dep,twenty_time_neg_index_dep,fifty_time_neg_index_dep,sl_label['dep_score'].values]
col1 = []
col2 = []
for i in range(5):
    tmpcol1 = [times[i]]*len(scores[i])
    if i==4:
        tmpcol2 = list(scores[i])
    else:
        tmpcol2 = list(sorted_neg_ids_scores_dep[scores[i],2])
    col1+=tmpcol1
    col2+=tmpcol2
dep_df['pnr'] = col1
dep_df['score'] = col2
dep_df['score'] = dep_df['score'].astype('float')
dep_df

In [ ]:

f,ax = plt.subplots(figsize=(7.5, 7.5))

sns.set_theme(style="white")
# Plot a histogram of the correlation
# sns.displot(data=exp_df,x='score', kind = 'kde',multiple='layer',hue = 'pnr', linewidth=0.5,alpha=.8)
# sns.kdeplot(data=np.asarray(sorted_neg_ids_scores_exp[one_time_neg_index_exp,2]), fill=True, legend=False, linewidth=0.5,alpha=.8,color=sns.color_palette("pastel")[0])
sns.kdeplot(data=dep_df,x='score',hue = 'pnr', fill=True,common_norm = False, linewidth=0.5,alpha=.6)
# sns.histplot(data=sl_corr, kde=True)

# plt.hist(sl_corr, bins=100)
# plt.xlabel('Gene expression correlation coefficient')
plt.xlabel('Gene dependency score correlation coefficient')
plt.ylabel('Frequency')
plt.title('Distribution of known SL gene pairs')
plt.xlim(-1,1)
# plt.show()

# plt.savefig('sl_corr_dep_03.svg')

# Negative sample base on gene expression

In [ ]:
meta_table = pd.read_csv('../data/preprocessed_data/meta_table_9845.csv')
meta_table

In [ ]:
len(set(meta_table['biogrid_id']))

In [ ]:
gene_id_dict = dict(zip(meta_table['symbol'],meta_table['unified_id']))
entrez_id_dict = dict(zip(meta_table['entrez_id'],meta_table['unified_id']))
gene_set = set(meta_table['symbol'])

In [ ]:
DepMap_Expression = pd.read_csv('/home/yimiaofeng/Projects/DepMap/OmicsExpressionProteinCodingGenesTPMLogp1.csv',header=0,index_col=0)
# DepMap_Expression.columns = [int(g[g.index('(')+1:-1]) for g in DepMap_Expression.columns]
DepMap_Expression.columns = [g[:g.index('(')-1] for g in DepMap_Expression.columns]
DepMap_Expression

In [ ]:
DepMap_Expression = DepMap_Expression[list(set(DepMap_Expression.columns)&(set(meta_table['symbol_in_kg'])|set(meta_table['symbol'])))]
DepMap_Expression

In [ ]:
gene_list =  list(set(DepMap_Expression.columns)&(set(meta_table['symbol_in_kg'])|set(meta_table['symbol'])))
symbol_in_kg_id = dict(zip(meta_table['symbol_in_kg'], meta_table['unified_id']))
symbol_id = dict(zip(meta_table['symbol'], meta_table['unified_id']))

In [ ]:
len(set([symbol_in_kg_id[g] for g in gene_list if g in symbol_in_kg_id.keys()])|set([symbol_id[g] for g in gene_list if g in symbol_id.keys()]))
all_ids = {}
for g in DepMap_Expression.columns:
    if g in symbol_in_kg_id.keys():
        all_ids[g] = symbol_in_kg_id[g]
        continue
    if g in symbol_id.keys():
        all_ids[g] = symbol_id[g]
        continue
print(len(all_ids))

In [ ]:
len(set(DepMap_Expression.columns.map(all_ids)))

In [ ]:
DepMap_Expression.columns = DepMap_Expression.columns.map(all_ids)
DepMap_Expression = DepMap_Expression[sorted(DepMap_Expression.columns)]
DepMap_Expression = DepMap_Expression.T
# DepMap_Expression = meta_table[['unified_id','symbol_in_kg']].merge(DepMap_Expression,left_on='symbol_in_kg',right_index=True,how='inner')
# DepMap_Expression = DepMap_Expression[DepMap_Expression.columns[2:]]
DepMap_Expression

In [ ]:
len(set(DepMap_Expression.index))

In [ ]:
DepMap_Expression_corr = DepMap_Expression.T.corr()

In [ ]:
DepMap_Expression_corr

In [ ]:
# sl_label = pd.read_csv('../data/preprocessed_data/human_sl_9845.csv')[['unified_id_A','unified_id_B']]
sl_label

In [ ]:
missing_gene = list(set(range(9845))-set(DepMap_Expression_corr.columns))
len(missing_gene)

In [ ]:
DepMap_Expression_corr.loc[:,missing_gene] = np.ones((DepMap_Expression_corr.shape[0],len(missing_gene)))
DepMap_Expression_corr = pd.concat([DepMap_Expression_corr,pd.DataFrame(np.ones((len(missing_gene),DepMap_Expression_corr.shape[1])),columns=DepMap_Expression_corr.columns,index=missing_gene)])
DepMap_Expression_corr

In [ ]:
DepMap_Expression_corr.to_csv('DepMap_Expression_corr.csv', float_format='%.6f')

In [ ]:
DepMap_Expression_corr = pd.read_csv('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/DepMap_Expression_corr.csv', index_col=0)
DepMap_Expression_corr

In [ ]:
DepMap_Expression_corr.columns = DepMap_Expression_corr.columns.astype(int)
DepMap_Expression_corr.index = DepMap_Expression_corr.index.astype(int)
DepMap_Expression_corr = DepMap_Expression_corr[range(9845)]
DepMap_Expression_corr = DepMap_Expression_corr.sort_index()
DepMap_Expression_corr

In [ ]:
all_one_ind = []
for i in range(9845):
    if all(DepMap_Expression_corr.loc[i,:]==1):
        all_one_ind.append(i)
for i in all_one_ind:
    DepMap_Expression_corr.loc[i,:] = np.nan
    DepMap_Expression_corr.loc[:,i] = np.nan

In [ ]:
DepMap_Expression_corr.loc[8822]

In [ ]:
DepMap_Expression_corr.columns = meta_table.loc[DepMap_Expression_corr.columns,'symbol'].values
DepMap_Expression_corr.index = meta_table.loc[DepMap_Expression_corr.index,'symbol'].values
DepMap_Expression_corr

In [ ]:
DepMap_Expression_corr = DepMap_Expression_corr.sort_index()
DepMap_Expression_corr = DepMap_Expression_corr.T.sort_index().T
DepMap_Expression_corr

In [ ]:
DepMap_Expression_corr.to_csv('DepMap_Expression_corr.csv', float_format='%.6f')

In [ ]:
sl_mat = sp.csc_matrix((np.ones(len(sl_label)), (sl_label['unified_id_A'], sl_label['unified_id_B'])), shape=(9845, 9845))
sl_mat = sl_mat+sl_mat.T
sl_mat = sl_mat.toarray()
neg_mat = np.ones((9845, 9845))
neg_wo_sl_mat = neg_mat - sl_mat
neg_wo_sl_up = np.triu(neg_wo_sl_mat,k=1)
neg_wo_sl_up = sp.csc_matrix(neg_wo_sl_up*DepMap_Expression_corr.values)
neg_ids = np.vstack(neg_wo_sl_up.nonzero()).T
neg_ids

In [ ]:
neg_ids.shape

In [ ]:
corr_scores = np.asarray(DepMap_Expression_corr.values)[neg_ids[:,0],neg_ids[:,1]]
neg_ids_scores = np.vstack([neg_ids.T,corr_scores]).T
neg_ids_scores
# corr_scores

In [ ]:
sorted_indices = np.argsort(neg_ids_scores[:, 2])
sorted_neg_ids_scores = neg_ids_scores[sorted_indices]
sorted_neg_ids_scores

In [ ]:
np.save('sorted_neg_ids_scores.npy',sorted_neg_ids_scores)

In [ ]:
import numpy as np
import pandas as pd
import copy

In [ ]:
sorted_neg_ids_scores_dep_wo_compt = np.load('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/wo_compt/sorted_neg_ids_scores_exp_wo_compt.npy')
sorted_neg_ids_scores_dep_wo_compt.shape

In [ ]:
sorted_neg_ids_scores = sorted_neg_ids_scores_dep_wo_compt

In [ ]:
contained_genes = set()
contained_neg_pairs_index = []
index_neg_pair = 0
while len(contained_genes) < 6445:# 6249: # 9828:
    if (len(set(sorted_neg_ids_scores[index_neg_pair, :2]) & contained_genes) < 2) and ~np.isnan(sorted_neg_ids_scores[index_neg_pair, 2]):
        contained_neg_pairs_index.append(index_neg_pair)
        contained_genes = contained_genes | set(sorted_neg_ids_scores[index_neg_pair, :2])
    index_neg_pair+=1

In [ ]:
# sorted_neg_ids_scores[contained_neg_pairs_index,:]
# len(contained_genes)
len(contained_neg_pairs_index)

In [ ]:
top5 = sorted_neg_ids_scores[contained_neg_pairs_index[-5:],:]
top5 = pd.DataFrame(top5,columns=['unified_id_A','unified_id_B','corr_score'])
top5['symbol_A'] = top5['unified_id_A'].map(meta_table.set_index('unified_id')['symbol'])
top5['symbol_B'] = top5['unified_id_B'].map(meta_table.set_index('unified_id')['symbol'])
top5

In [ ]:
len(contained_neg_pairs_index)

In [ ]:
one_time_neg_index = copy.deepcopy(contained_neg_pairs_index)
five_time_neg_index = copy.deepcopy(contained_neg_pairs_index)
twinty_time_neg_index = copy.deepcopy(contained_neg_pairs_index)
fifty_time_neg_index = copy.deepcopy(contained_neg_pairs_index)

one_time_neg_index.extend(list(range(len(sl_label)-len(contained_neg_pairs_index))))
one_time_neg_index = set(one_time_neg_index)
index_neg_pair = len(sl_label)
while len(one_time_neg_index) < len(sl_label):
    one_time_neg_index.add(index_neg_pair)
    index_neg_pair+=1

five_time_neg_index.extend(list(range(len(sl_label)*5-len(contained_neg_pairs_index))))
five_time_neg_index = set(five_time_neg_index)
index_neg_pair = len(sl_label)*5
while len(five_time_neg_index) < len(sl_label)*5:
    five_time_neg_index.add(index_neg_pair)
    index_neg_pair+=1

twinty_time_neg_index.extend(list(range(len(sl_label)*20-len(contained_neg_pairs_index))))
twinty_time_neg_index = set(twinty_time_neg_index)
index_neg_pair = len(sl_label)*20
while len(twinty_time_neg_index) < len(sl_label)*20:
    twinty_time_neg_index.add(index_neg_pair)
    index_neg_pair+=1

fifty_time_neg_index.extend(list(range(len(sl_label)*50-len(contained_neg_pairs_index))))
fifty_time_neg_index = set(fifty_time_neg_index)
index_neg_pair = len(sl_label)*50
while len(fifty_time_neg_index) < len(sl_label)*50:
    fifty_time_neg_index.add(index_neg_pair)
    index_neg_pair+=1

In [ ]:
np.save('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/wo_compt/one_time_neg_index_exp_wo_compt.npy', list(one_time_neg_index))
np.save('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/wo_compt/five_time_neg_index_exp_wo_compt.npy', list(five_time_neg_index))
np.save('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/wo_compt/twenty_time_neg_index_exp_wo_compt.npy', list(twinty_time_neg_index))
np.save('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/wo_compt/fifty_time_neg_index_exp_wo_compt.npy', list(fifty_time_neg_index))

In [ ]:
sorted_neg_ids_scores[index_neg_pair,:]

In [ ]:
len(set(one_time[:,0])|set(one_time[:,1]))

In [ ]:
sl_corr = np.zeros((len(sl_label),1))
not_in = 0
for i in range(len(sl_label)):
    a, b = sl_label.iloc[i,:]
    sl_corr[i] = DepMap_Expression_corr.loc[a,b]
    if sl_corr[i] == 1:
        sl_corr[i] = 2

# Plot a histogram of the correlation
# sns.histplot(data=sl_corr)

# plt.hist(sl_corr, bins=100)
# plt.xlabel('Value')
# plt.ylabel('Frequency')                                                                                                                                                                                                                                 
# plt.title('Histogram of List Values')
# plt.show()

In [ ]:
f,ax = plt.subplots(figsize=(7.5, 7.5))

# Plot a histogram of the correlation
sns.kdeplot(data=sl_label['exp_score'], fill=True, legend=False, linewidth=0.5,palette="pastel",alpha=.8)
# sns.histplot(data=sl_corr, kde=True)

# plt.hist(sl_corr, bins=100)
plt.xlabel('Gene expression correlation coefficient')
plt.ylabel('Frequency')
plt.title('Distribution of known SL gene pairs')
plt.xlim(-1,1)
# plt.show()
# plt.savefig('sl_corr_exp_18.svg')

In [ ]:
f,ax = plt.subplots(figsize=(7.5, 7.5))

# Plot a histogram of the correlation
sns.kdeplot(data=sl_corr, fill=True, legend=False, linewidth=0.5,palette="pastel",alpha=.8)
# sns.histplot(data=sl_corr, kde=True)

# plt.hist(sl_corr, bins=100)
plt.xlabel('Gene expression correlation coefficient')
plt.ylabel('Frequency')
plt.title('Distribution of known SL gene pairs')
plt.xlim(-1,1)
# plt.show()
plt.savefig('sl_corr_exp_18.svg')

# Negative sample base on gene dependency score

In [ ]:
DepMap_Effect = pd.read_csv('/home/yimiaofeng/Projects/DepMap/CRISPRGeneEffect.csv',header = 0,index_col=0)
DepMap_Effect.columns = [g[:g.index('(')-1] for g in DepMap_Effect.columns]
DepMap_Effect

In [ ]:
DepMap_Effect = DepMap_Effect[list(set(DepMap_Effect.columns)&(set(meta_table['symbol_in_kg'])|set(meta_table['symbol'])))]
DepMap_Effect

In [ ]:
len(set(DepMap_Effect.columns))

In [ ]:
gene_list =  list(set(DepMap_Effect.columns)&(set(meta_table['symbol_in_kg'])|set(meta_table['symbol'])))
symbol_in_kg_id = dict(zip(meta_table['symbol_in_kg'], meta_table['unified_id']))
symbol_id = dict(zip(meta_table['symbol'], meta_table['unified_id']))

In [ ]:
len(set([symbol_in_kg_id[g] for g in gene_list if g in symbol_in_kg_id.keys()])|set([symbol_id[g] for g in gene_list if g in symbol_id.keys()]))
all_ids = {}
for g in DepMap_Effect.columns:
    if g in symbol_in_kg_id.keys():
        all_ids[g] = symbol_in_kg_id[g]
        continue
    if g in symbol_id.keys():
        all_ids[g] = symbol_id[g]
        continue
print(len(all_ids))

In [ ]:
DepMap_Effect.columns = DepMap_Effect.columns.map(all_ids)
DepMap_Effect = DepMap_Effect[sorted(DepMap_Effect.columns)]
DepMap_Effect = DepMap_Effect.T
DepMap_Effect

In [ ]:
DepMap_Effect = DepMap_Effect.T.corr()
DepMap_Effect

In [ ]:
missing_gene = list(set(range(9845))-set(DepMap_Effect.columns))
len(missing_gene)

In [ ]:
DepMap_Effect.loc[:,missing_gene] = np.ones((DepMap_Effect.shape[0],len(missing_gene)))
DepMap_Effect = pd.concat([DepMap_Effect,pd.DataFrame(np.ones((len(missing_gene),DepMap_Effect.shape[1])),columns=DepMap_Effect.columns,index=missing_gene)])
DepMap_Effect

In [ ]:
DepMap_Effect.loc[:,missing_gene] = np.ones((DepMap_Effect.shape[0],len(missing_gene)))*np.nan
DepMap_Effect.loc[missing_gene,:] = np.ones((len(missing_gene),DepMap_Effect.shape[1]))*np.nan
DepMap_Effect

In [ ]:
DepMap_Effect_corr = DepMap_Effect[range(9845)]
DepMap_Effect_corr = DepMap_Effect_corr.sort_index()
DepMap_Effect_corr

In [ ]:
DepMap_Effect_corr.to_csv('../data/preprocessed_data/DepMap_Effect_corr.csv', float_format='%.6f')

In [ ]:
..DepMap_Effect_corr = pd.read_csv('../data/preprocessed_data/DepMap_Effect_corr.csv',index_col=0)
DepMap_Effect_corr

In [ ]:
DepMap_Effect_corr.columns = DepMap_Effect_corr.columns.astype(int)
DepMap_Effect_corr.index = DepMap_Effect_corr.index.astype(int)
DepMap_Effect_corr.columns = meta_table.loc[DepMap_Effect_corr.columns,'symbol'].values
DepMap_Effect_corr.index = meta_table.loc[DepMap_Effect_corr.index,'symbol'].values
DepMap_Effect_corr = DepMap_Effect_corr.sort_index()
DepMap_Effect_corr = DepMap_Effect_corr.T.sort_index().T
DepMap_Effect_corr

In [ ]:
sl_corr = np.zeros((len(sl_label),1))
not_in = 0
for i in range(len(sl_label)):
    a, b = sl_label.iloc[i,:]
    if DepMap_Effect_corr.loc[a,b] == np.nan:
        sl_corr[i] = 2
    else:
        sl_corr[i] = DepMap_Effect_corr.loc[a,b]
        # sl_corr[i] = abs(DepMap_Effect_corr.loc[a,b])

# # Plot a histogram of the correlation
# plt.hist(sl_corr, bins=100)
# plt.xlabel('Value')
# plt.ylabel('Frequency')                                                                                                                                                                                                                                 
# plt.title('Histogram of List Values')
# plt.show()

In [ ]:
f,ax = plt.subplots(figsize=(7.5, 7.5))

sns.set_theme(style="white")
# Plot a histogram of the correlation
sns.kdeplot(data=sl_corr, fill=True, legend=False, linewidth=0.5,alpha=.8,color=sns.color_palette("pastel")[0])
# sns.histplot(data=sl_corr, kde=True)

# plt.hist(sl_corr, bins=100)
plt.xlabel('Gene dependency score correlation coefficient')
plt.ylabel('Frequency')
plt.title('Distribution of known SL gene pairs')
plt.xlim(-1,1)
# plt.show()

plt.savefig('sl_corr_dep_18.svg')

In [ ]:
sns.color_palette('pastel')[2]

In [ ]:
sl_mat = sp.csc_matrix((np.ones(len(sl_label)), (sl_label['unified_id_A'], sl_label['unified_id_B'])), shape=(9845, 9845))
sl_mat = sl_mat+sl_mat.T
sl_mat = sl_mat.toarray()
neg_mat = np.ones((9845, 9845))
neg_wo_sl_mat = neg_mat - sl_mat
neg_wo_sl_up = np.triu(neg_wo_sl_mat,k=1)
neg_wo_sl_up = sp.csc_matrix(neg_wo_sl_up*abs(DepMap_Effect_corr.values))
neg_ids = np.vstack(neg_wo_sl_up.nonzero()).T
corr_scores = np.asarray(abs(DepMap_Effect_corr.values))[neg_ids[:,0],neg_ids[:,1]]
neg_ids_scores = np.vstack([neg_ids.T,corr_scores]).T
sorted_indices = np.argsort(neg_ids_scores[:, 2])[::-1]
sorted_neg_ids_scores = neg_ids_scores[sorted_indices]
sorted_neg_ids_scores

In [ ]:
sorted_neg_ids_scores = sorted_neg_ids_scores[~np.isnan(sorted_neg_ids_scores[:,2])]

In [ ]:
np.save('sorted_neg_ids_scores_dep.npy',sorted_neg_ids_scores)

In [ ]:
contained_genes = set()
contained_neg_pairs_index = []
index_neg_pair = 0
while len(contained_genes) < 9529:
    if len(set(sorted_neg_ids_scores[index_neg_pair, :2]) & contained_genes) < 2:
        contained_neg_pairs_index.append(index_neg_pair)
        contained_genes = contained_genes | set(sorted_neg_ids_scores[index_neg_pair, :2])
    index_neg_pair+=1

In [ ]:
# sorted_neg_ids_scores[index_neg_pair,:]
top5 = sorted_neg_ids_scores[contained_neg_pairs_index[-5:],:]
top5 = pd.DataFrame(top5,columns=['unified_id_A','unified_id_B','corr_score'])
top5['symbol_A'] = top5['unified_id_A'].map(meta_table.set_index('unified_id')['symbol'])
top5['symbol_B'] = top5['unified_id_B'].map(meta_table.set_index('unified_id')['symbol'])
top5

In [ ]:
contained_neg_pairs_index[:5]
len(sorted_neg_ids_scores)

In [ ]:
one_time_neg_index = copy.deepcopy(contained_neg_pairs_index)
index_neg_pair = 0
while len(one_time_neg_index) < len(sl_label):
    if index_neg_pair not in contained_neg_pairs_index:
        one_time_neg_index.append(index_neg_pair)
    index_neg_pair+=1
print(len(one_time_neg_index))
# index_neg_pair = 0
five_time_neg_index = copy.deepcopy(one_time_neg_index)
while len(five_time_neg_index) < len(sl_label)*5:
    if index_neg_pair not in contained_neg_pairs_index:
        five_time_neg_index.append(index_neg_pair)
    index_neg_pair+=1
print(len(five_time_neg_index))
# index_neg_pair = 0
twinty_time_neg_index = copy.deepcopy(five_time_neg_index)
while len(twinty_time_neg_index) < len(sl_label)*20:
    if index_neg_pair not in contained_neg_pairs_index:
        twinty_time_neg_index.append(index_neg_pair)
    index_neg_pair+=1
print(len(twinty_time_neg_index))
# index_neg_pair = 0
fifty_time_neg_index = copy.deepcopy(twinty_time_neg_index)
while len(fifty_time_neg_index) < len(sl_label)*50:
    if index_neg_pair not in contained_neg_pairs_index:
        fifty_time_neg_index.append(index_neg_pair)
    index_neg_pair+=1
print(len(fifty_time_neg_index))

In [ ]:
np.save('one_time_neg_index_dep.npy', one_time_neg_index)
np.save('five_time_neg_index_dep.npy', five_time_neg_index)
np.save('twenty_time_neg_index_dep.npy', twinty_time_neg_index)
np.save('fifty_time_neg_index_dep.npy', fifty_time_neg_index)

In [ ]:
sorted_neg_ids_scores[one_time_neg_index,:2]

In [ ]:
import os
import sys
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, trange
import scipy.sparse as sp

In [ ]:
slpair = pd.read_csv('../data/preprocessed_data/human_sl_9845.csv')
negexp_ind = np.load('../data/preprocessed_data/five_time_neg_index_exp.npy')
negexp_pair = np.load('../data/preprocessed_data/sorted_neg_ids_scores_exp.npy')


In [ ]:
negexp_pair

In [ ]:
slpair

In [ ]:
count=0
neg_ind = []
pos_pairset = set(zip(slpair['unified_id_A'],slpair['unified_id_B']))
i=0
while count<len(slpair):
    if ((negexp_pair[i,0],negexp_pair[i,1]) not in pos_pairset) and ((negexp_pair[i,1],negexp_pair[i,0]) not in pos_pairset):
        count+=1
        neg_ind.append(i)
    i+=1

In [ ]:
train_pos_mat = sp.csc_matrix((np.ones(len(slpair)), (slpair['unified_id_A'], slpair['unified_id_B'])), shape=(9845, 9845))
train_pos_mat = train_pos_mat+train_pos_mat.T
train_pos_mat

In [ ]:
train_neg_mat = sp.csc_matrix((np.ones(len(neg_ind)), (negexp_pair[neg_ind,0], negexp_pair[neg_ind,1])), shape=(9845, 9845))
train_neg_mat = train_neg_mat+train_neg_mat.T
train_neg_mat

In [ ]:
pos_samples = [[train_pos_mat],[train_pos_mat],[slpair[['unified_id_A','unified_id_B']].values.astype(int)],[slpair[['unified_id_A','unified_id_B']].values.astype(int)]]
neg_samples = [[train_neg_mat],[train_neg_mat],[np.asarray((negexp_pair[neg_ind,0], negexp_pair[neg_ind,1])).astype(int).T],[np.asarray((negexp_pair[neg_ind,0], negexp_pair[neg_ind,1])).astype(int).T]]

In [ ]:
np.asarray([pos_samples,neg_samples]).shape

In [ ]:
np.save('../data/data_split/SLMGAE_fin_res.npy',[pos_samples,neg_samples])
np.save('../data/data_split/DDGCN_fin_res.npy',[pos_samples,neg_samples])
np.save('../data/data_split/GCATSL_fin_res.npy',[pos_samples,neg_samples])
np.save('../data/data_split/GRSMF_fin_res.npy',[pos_samples,neg_samples])
np.save('../data/data_split/KG4SL_fin_res.npy',[pos_samples,neg_samples])
np.save('../data/data_split/MGE4SL_fin_res.npy',[pos_samples,neg_samples])
np.save('../data/data_split/PiLSL_fin_res.npy',[pos_samples,neg_samples])
np.save('../data/data_split/PTGNN_fin_res.npy',[pos_samples,neg_samples])

In [ ]:
import random

In [ ]:
pos_position = np.asarray(slpair[['unified_id_A', 'unified_id_B']].values)
for i in range(len(pos_position)):
    if pos_position[i, 0] > pos_position[i, 1]:
        pos_position[i, 0], pos_position[i, 1] = pos_position[i, 1], pos_position[i, 0]

neg_position = np.asarray(np.ones((9845, 9845)), dtype='bool')

neg_position[pos_position[:, 1], pos_position[:, 0]] = False
neg_position[pos_position[:, 0], pos_position[:, 1]] = False
neg_position = np.triu(neg_position, k = 1)
neg_position = np.where(neg_position == True)
neg_position = np.vstack(neg_position).transpose()

init_pos_index = list(range(len(pos_position)))
init_neg_index = list(range(len(neg_position)))

training_size = len(init_pos_index)
print(training_size)

random.seed(123)
np.random.seed(123)
pos_position = pos_position[random.sample(init_pos_index, training_size)]
neg_position = neg_position[random.sample(init_neg_index, training_size)]

In [ ]:
train_neg_mat = sp.csc_matrix((np.ones(len(neg_position)), (neg_position[:,0], neg_position[:,1])), shape=(9845, 9845))
train_neg_mat = train_neg_mat+train_neg_mat.T
train_neg_mat

In [ ]:
pos_samples = [[train_pos_mat],[train_pos_mat],[slpair[['unified_id_A','unified_id_B']].values.astype(int)],[slpair[['unified_id_A','unified_id_B']].values.astype(int)]]
neg_samples = [[train_neg_mat],[train_neg_mat],[np.asarray((neg_position[:,0], neg_position[:,1])).astype(int).T],[np.asarray((neg_position[:,0], neg_position[:,1])).astype(int).T]]

In [ ]:
np.asarray([pos_samples,neg_samples]).shape

In [ ]:
np.save('../data/data_split/NSF4SL_fin_res.npy',[pos_samples,neg_samples])
np.save('../data/data_split/CMFW_fin_res.npy',[pos_samples,neg_samples])
np.save('../data/data_split/SL2MF_fin_res.npy',[pos_samples,neg_samples])

In [ ]:
# pos,neg
# mat train mat test ind train med test

In [ ]:
data[0][2][4].shape

In [ ]:
data[1][3][4].shape